In [ ]:
import pandas as pd
from google.cloud import bigquery
import google.auth
import warnings

# Use default Google Cloud authentication
credentials, project = google.auth.default()
client = bigquery.Client(credentials=credentials, project=project)

# Define datasets
datasets = {
    "World Bank International Education": "bigquery-public-data.world_bank_intl_education.international_education",
    "Chicago Taxi Trips": "bigquery-public-data.chicago_taxi_trips.taxi_trips",
    "Stack Overflow Data": "bigquery-public-data.stackoverflow.posts_questions"
}

# Define queries
queries = {
    "-- Query 1: Countries spending the most on education (as % of GDP)": """
        SELECT country_name, AVG(value) AS avg_spending
        FROM `bigquery-public-data.world_bank_intl_education.international_education`
        WHERE indicator_code = 'SE.XPD.TOTL.GD.ZS' AND year >= 2010
        GROUP BY country_name
        HAVING avg_spending > 5
        ORDER BY avg_spending DESC;
    """,
    "-- Query 2: Busiest taxi pickup locations in Chicago": """
        SELECT pickup_community_area, COUNT(*) AS num_pickups
        FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
        WHERE pickup_community_area IS NOT NULL
        GROUP BY pickup_community_area
        ORDER BY num_pickups DESC
        LIMIT 10;
    """,
    "-- Query 3: Average taxi speed per hour in Chicago": """
        WITH RelevantRides AS (
            SELECT EXTRACT(HOUR FROM trip_start_timestamp) AS hour,
                   COUNT(*) AS num_trips,
                   3600 * SUM(trip_miles) / SUM(trip_seconds) AS avg_mph
            FROM `bigquery-public-data.chicago_taxi_trips.taxi_trips`
            WHERE EXTRACT(MONTH FROM trip_start_timestamp) BETWEEN 2 AND 3
                  AND trip_seconds > 0 AND trip_miles > 0
            GROUP BY hour
        )
        SELECT hour, num_trips, avg_mph
        FROM RelevantRides
        ORDER BY hour;
    """,

    "-- Query 4: Find top users who answered questions tagged 'bigquery'": """
        SELECT pa.owner_user_id AS user_id, COUNT(*) AS number_of_answers
        FROM `bigquery-public-data.stackoverflow.posts_answers` AS pa
        INNER JOIN `bigquery-public-data.stackoverflow.posts_questions` AS q
        ON pa.parent_id = q.id
        WHERE q.tags LIKE '%bigquery%'
        GROUP BY pa.owner_user_id
        ORDER BY number_of_answers DESC
        LIMIT 10;
    """,

    # detailed description of how I created this 'Query 5' in the file 'queries.sql'
    "-- Query 5: Find the fastest-growing programming languages on Stack Overflow": """
        WITH yearly_tag_counts AS (
        SELECT tag_name, 
               EXTRACT(YEAR FROM creation_date) AS year, 
               COUNT(*) AS tag_count
        FROM `bigquery-public-data.stackoverflow.posts_questions`, 
        UNNEST(SPLIT(tags, '|')) AS tag_name
        GROUP BY tag_name, year
        ),
        growth_rate AS (
        SELECT 
            tag_name,
            MAX(tag_count) AS max_count,
            MIN(tag_count) AS min_count,
            MAX(tag_count) - MIN(tag_count) AS growth,
            MIN(year) AS first_year,
            MAX(year) AS last_year
        FROM yearly_tag_counts
        GROUP BY tag_name
        )
        SELECT 
            tag_name, 
            growth, 
            first_year, 
            last_year,
        ROUND(SAFE_DIVIDE(growth, NULLIF(min_count, 0)), 2) AS growth_ratio
        FROM growth_rate
        WHERE growth > 5000  -- Filter for significant growth
        ORDER BY growth_ratio DESC
        LIMIT 10;
    """,
}

# Execute and display each dataset followed by its related queries
for dataset_name, dataset in datasets.items():
    print(f"\nOriginal Dataset: {dataset_name}\n")
    query_job = client.query(f"SELECT * FROM `{dataset}` LIMIT 5")
    dataset_preview = query_job.to_dataframe()
    display(dataset_preview)
    
    # Execute related queries
    for title, query in queries.items():
        if dataset.split(".")[-1] in query:
            print(f"\nRunning query: {title}\n")
            query_job = client.query(query)
            results = query_job.to_dataframe()
            display(results)


Original Dataset: World Bank International Education



,country_name,country_code,indicator_name,indicator_code,value,year
0,Chad,TCD,"Enrolment in lower secondary education, both s...",UIS.E.2,321921.0,2012
1,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,68809.0,2006
2,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,30551.0,1999
3,Chad,TCD,"Enrolment in upper secondary education, both s...",UIS.E.3,79784.0,2007
4,Chad,TCD,"Repeaters in primary education, all grades, bo...",UIS.R.1,282699.0,2006



Running query: -- Query 1: Countries spending the most on education (as % of GDP)



,country_name,avg_spending
0,Cuba,12.837270
1,"Micronesia, Fed. Sts.",12.467750
2,Solomon Islands,10.001080
3,Moldova,8.372153
4,Namibia,8.349610
...,...,...
57,Mexico,5.109818
58,Belarus,5.085277
59,St. Vincent and the Grenadines,5.085230
60,Switzerland,5.031946



Original Dataset: Chicago Taxi Trips



...

,id,title,body,accepted_answer_id,answer_count,comment_count,community_owned_date,creation_date,favorite_count,last_activity_date,last_edit_date,last_editor_display_name,last_editor_user_id,owner_display_name,owner_user_id,parent_id,post_type_id,score,tags,view_count
0,71102973,Product-card snippet not loading images - Shopify,<p>Hi there i'm making some changes at my prod...,<NA>,0,0,NaT,2022-02-13 17:00:30.403000+00:00,<NA>,2022-02-13 17:00:30.403000+00:00,NaT,None,<NA>,None,18184792,None,1,0,shopify|liquid|code-snippets,256
1,71104461,Google Sign In One Tap & Laravel Socialite - B...,<p>I am trying to set up Google Sign In One Ta...,<NA>,0,0,NaT,2022-02-13 20:11:45.200000+00:00,<NA>,2022-02-13 20:17:01.387000+00:00,2022-02-13 20:17:01.387000+00:00,None,3922429,None,3922429,None,1,0,laravel|oauth-2.0,256
2,71110679,Robot Framework - how to get names of all keyw...,<p>I'm currently working on reporting of Robot...,<NA>,0,0,NaT,2022-02-14 10:51:58.633000+00:00,<NA>,2022-02-14 10:51:58.633000+00:00,NaT,None,<NA>,None,14633035,None,1,0,robotframework|reporting,256
3,71122941,Chrome DevTools debugger is slow when debuggin...,<p>DevTools Chrome (and Edge) debugger steps a...,<NA>,0,0,NaT,2022-02-15 08:06:43.857000+00:00,<NA>,2022-02-15 08:06:43.857000+00:00,NaT,None,<NA>,None,12442081,None,1,0,reactjs|google-chrome|debugging|knockout.js|go...,256
4,71134241,Pyomo: how to work with 2D arrays or Sets,<p>I have a matrix that carries different inte...,71134746,1,0,NaT,2022-02-15 22:33:31.777000+00:00,<NA>,2022-02-15 23:38:27.650000+00:00,NaT,None,<NA>,None,18190284,None,1,0,python|matrix|optimization|set|pyomo,256



Running query: -- Query 4: Find top users who answered questions tagged 'bigquery'



,user_id,number_of_answers
0,5221944,5203
1,1144035,1634
2,132438,898
3,6253347,737
4,1366527,620
5,243782,613
6,13473525,380
7,4490873,275
8,11206202,264
9,2877278,261



Running query: -- Query 5: Find the fastest-growing programming languages on Stack Overflow



,tag_name,growth,first_year,last_year,growth_ratio
0,angularjs,67663,2009,2022,67663.0
1,pandas,50335,2010,2022,50335.0
2,typescript,34545,2011,2022,34545.0
3,angular,55998,2013,2022,27999.0
4,dataframe,25715,2008,2022,25715.0
5,spring-boot,23329,2010,2022,23329.0
6,firebase,22469,2010,2022,22469.0
7,docker,20531,2010,2022,20531.0
8,flutter,40690,2012,2022,20345.0
9,twitter-bootstrap,20014,2009,2022,20014.0
